<a href="https://colab.research.google.com/github/Zebo616/Attack-Lab/blob/master/Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB

Here we read in the data.

We will use three different data sets: the data set with only the original features, the data set with only the feature engineered "team sum/average" values previously computed, and a third dataset with both of these.

In [6]:
# the entire dataset
data = pd.read_csv("3000_engineered_data.csv", header=0)
data.head()

,b1_champ_id,b2_champ_id,b3_champ_id,b4_champ_id,b5_champ_id,r6_champ_id,r7_champ_id,r8_champ_id,r9_champ_id,r10_champ_id,b1_win_ratio,b2_win_ratio,b3_win_ratio,b4_win_ratio,b5_win_ratio,r6_win_ratio,r7_win_ratio,r8_win_ratio,r9_win_ratio,r10_win_ratio,b1_games_played,b2_games_played,b3_games_played,b4_games_played,b5_games_played,r6_games_played,r7_games_played,r8_games_played,r9_games_played,r10_games_played,b1_avg_gold,b2_avg_gold,b3_avg_gold,b4_avg_gold,b5_avg_gold,r6_avg_gold,r7_avg_gold,r8_avg_gold,r9_avg_gold,r10_avg_gold,...,r6_avg_kda,r7_avg_kda,r8_avg_kda,r9_avg_kda,r10_avg_kda,b1_avg_cs,b2_avg_cs,b3_avg_cs,b4_avg_cs,b5_avg_cs,r6_avg_cs,r7_avg_cs,r8_avg_cs,r9_avg_cs,r10_avg_cs,b1_champ_win_rate,b2_champ_win_rate,b3_champ_win_rate,b4_champ_win_rate,b5_champ_win_rate,r6_champ_win_rate,r7_champ_win_rate,r8_champ_win_rate,r9_champ_win_rate,r10_champ_win_rate,b_win_sum,r_win_sum,b_avg_gold,r_avg_gold,b_avg_cs,r_avg_cs,b_avg_kda,r_avg_kda,b_games_played,r_games_played,b_champ_win_sum,r_champ_win_sum,b_champ_player_win_diff,r_champ_player_win_diff,winner
0,34,203,99,81,98,91,121,31,15,16,0.666667,0.666667,0.468085,0.576923,0.535865,0.637500,0.631579,0.625000,0.567164,0.470588,3,6,94,52,0,80,19,16,67,17,16383.666667,11644.000000,9459.000000,11004.846154,9627.183046,12691.262500,10456.105263,12065.875000,11399.000000,8485.058824,...,3.662375,1.974211,2.684375,3.710000,2.359412,211.333333,157.300000,45.200000,151.830769,108.702305,166.610000,118.242105,122.800000,175.200000,11.729412,0.546514,0.476829,0.507657,0.457430,0.536239,0.547912,0.505691,0.531711,0.557449,0.518675,2.914206,2.931831,58118.695866,55097.301587,674.366408,594.581517,13.298621,14.390372,155,199,2.524669,2.661438,0.389538,0.270394,r
1,23,67,234,141,99,498,25,142,27,111,1.000000,0.200000,0.451327,0.640000,0.468085,0.510301,1.000000,0.500000,0.405070,0.559524,2,5,113,25,94,0,2,4,0,168,12829.000000,8496.200000,10901.920354,9336.400000,9459.000000,11914.683142,13946.000000,10404.250000,9439.219577,8249.000000,...,3.007855,2.270000,2.645000,1.690821,2.690000,200.000000,123.420000,120.887611,34.512000,45.200000,165.252596,42.500000,90.025000,133.869469,45.000000,0.553178,0.543811,0.506401,0.510037,0.507657,0.514187,0.549502,0.481565,0.434316,0.554777,2.759413,2.974894,51022.520354,53953.152718,524.019611,476.647065,10.266106,12.303675,239,174,2.621083,2.534347,0.138329,0.440548,b
2,99,222,23,37,82,17,154,63,51,62,0.517241,0.571429,0.506494,0.580645,0.529355,0.512472,0.559418,0.400000,0.600000,0.420000,29,21,77,31,0,0,0,35,30,0,8990.000000,14413.952381,13485.000000,7853.096774,11216.128205,11212.936873,9870.395100,7789.914286,12636.000000,9072.000000,...,2.076204,6.397587,1.534286,4.366000,2.290000,78.303448,190.771429,191.700000,12.580645,142.820892,141.928393,97.939132,47.262857,165.820000,105.200000,0.507657,0.537599,0.553178,0.508135,0.533896,0.513674,0.552919,0.554749,0.514967,0.420000,2.705163,2.491890,55958.177360,50581.246258,616.176414,558.150382,12.804336,16.664077,158,65,2.640465,2.556310,0.064698,-0.064419,r
3,122,119,45,5,103,83,25,202,268,77,0.497004,0.650000,0.375000,1.000000,0.461538,0.587535,0.500000,0.666667,0.482143,1.000000,0,40,16,1,26,0,6,3,56,2,11181.465808,13709.000000,8652.000000,10009.000000,11832.384615,11680.469576,9368.666667,10773.000000,10731.125000,9592.000000,...,2.362406,3.178333,1.890000,2.022321,1.170000,143.656231,174.800000,53.300000,27.000000,137.484615,175.621208,31.000000,112.000000,170.705357,15.000000,0.502224,0.520850,0.487365,0.574638,0.530157,0.581295,0.549502,0.534400,0.422799,0.534542,2.983542,3.236344,55383.850423,52145.261243,536.240847,504.326565,22.197512,10.623061,83,67,2.615234,2.622539,0.368309,0.613805,r
4,75,25,62,19,202,86,119,18,43,11,0.554367,0.333333,0.420000,0.600000,0.500000,0.000000,0.000000,0.650000,0.000000,0.545451,0,9,0,5,38,1,1,20,1,0,11401.275223,10597.444444,9072.000000,13082.000000,10994.000000,12009.000000,9005.000000,13383.000000,11146.000000,11850.221581,...,1.880000,2.570000,2.050000,2.2700

In [7]:
# only feature engineered items
data_teams = data.iloc[:,-15:]
data_teams.head()

,b_win_sum,r_win_sum,b_avg_gold,r_avg_gold,b_avg_cs,r_avg_cs,b_avg_kda,r_avg_kda,b_games_played,r_games_played,b_champ_win_sum,r_champ_win_sum,b_champ_player_win_diff,r_champ_player_win_diff,winner
0,2.914206,2.931831,58118.695866,55097.301587,674.366408,594.581517,13.298621,14.390372,155,199,2.524669,2.661438,0.389538,0.270394,r
1,2.759413,2.974894,51022.520354,53953.152718,524.019611,476.647065,10.266106,12.303675,239,174,2.621083,2.534347,0.138329,0.440548,b
2,2.705163,2.491890,55958.177360,50581.246258,616.176414,558.150382,12.804336,16.664077,158,65,2.640465,2.556310,0.064698,-0.064419,r
3,2.983542,3.236344,55383.850423,52145.261243,536.240847,504.326565,22.197512,10.623061,83,67,2.615234,2.622539,0.368309,0.613805,r
4,2.407700,1.195451,55146.719667,57393.221581,522.544539,559.387012,11.899758,11.197233,52,23,2.587983,2.648679,-0.180283,-1.453228,b


In [8]:
# the original features data set
data_original = data.iloc[:,:-15]
data_original['winner'] = data.iloc[:,-1]
data_original.head()

,b1_champ_id,b2_champ_id,b3_champ_id,b4_champ_id,b5_champ_id,r6_champ_id,r7_champ_id,r8_champ_id,r9_champ_id,r10_champ_id,b1_win_ratio,b2_win_ratio,b3_win_ratio,b4_win_ratio,b5_win_ratio,r6_win_ratio,r7_win_ratio,r8_win_ratio,r9_win_ratio,r10_win_ratio,b1_games_played,b2_games_played,b3_games_played,b4_games_played,b5_games_played,r6_games_played,r7_games_played,r8_games_played,r9_games_played,r10_games_played,b1_avg_gold,b2_avg_gold,b3_avg_gold,b4_avg_gold,b5_avg_gold,r6_avg_gold,r7_avg_gold,r8_avg_gold,r9_avg_gold,r10_avg_gold,b1_avg_kda,b2_avg_kda,b3_avg_kda,b4_avg_kda,b5_avg_kda,r6_avg_kda,r7_avg_kda,r8_avg_kda,r9_avg_kda,r10_avg_kda,b1_avg_cs,b2_avg_cs,b3_avg_cs,b4_avg_cs,b5_avg_cs,r6_avg_cs,r7_avg_cs,r8_avg_cs,r9_avg_cs,r10_avg_cs,b1_champ_win_rate,b2_champ_win_rate,b3_champ_win_rate,b4_champ_win_rate,b5_champ_win_rate,r6_champ_win_rate,r7_champ_win_rate,r8_champ_win_rate,r9_champ_win_rate,r10_champ_win_rate,winner
0,34,203,99,81,98,91,121,31,15,16,0.666667,0.666667,0.468085,0.576923,0.535865,0.637500,0.631579,0.625000,0.567164,0.470588,3,6,94,52,0,80,19,16,67,17,16383.666667,11644.000000,9459.000000,11004.846154,9627.183046,12691.262500,10456.105263,12065.875000,11399.000000,8485.058824,2.760000,2.510000,2.200000,2.52,3.308621,3.662375,1.974211,2.684375,3.710000,2.359412,211.333333,157.300000,45.200000,151.830769,108.702305,166.610000,118.242105,122.800000,175.200000,11.729412,0.546514,0.476829,0.507657,0.457430,0.536239,0.547912,0.505691,0.531711,0.557449,0.518675,r
1,23,67,234,141,99,498,25,142,27,111,1.000000,0.200000,0.451327,0.640000,0.468085,0.510301,1.000000,0.500000,0.405070,0.559524,2,5,113,25,94,0,2,4,0,168,12829.000000,8496.200000,10901.920354,9336.400000,9459.000000,11914.683142,13946.000000,10404.250000,9439.219577,8249.000000,1.800000,1.040000,2.486106,2.74,2.200000,3.007855,2.270000,2.645000,1.690821,2.690000,200.000000,123.420000,120.887611,34.512000,45.200000,165.252596,42.500000,90.025000,133.869469,45.000000,0.553178,0.543811,0.506401,0.510037,0.507657,0.514187,0.549502,0.481565,0.434316,0.554777,b
2,99,222,23,37,82,17,154,63,51,62,0.517241,0.571429,0.506494,0.580645,0.529355,0.512472,0.559418,0.400000,0.600000,0.420000,29,21,77,31,0,0,0,35,30,0,8990.000000,14413.952381,13485.000000,7853.096774,11216.128205,11212.936873,9870.395100,7789.914286,12636.000000,9072.000000,2.177931,3.620000,1.390000,3.50,2.116405,2.076204,6.397587,1.534286,4.366000,2.290000,78.303448,190.771429,191.700000,12.580645,142.820892,141.928393,97.939132,47.262857,165.820000,105.200000,0.507657,0.537599,0.553178,0.508135,0.533896,0.513674,0.552919,0.554749,0.514967,0.420000,r
3,122,119,45,5,103,83,25,202,268,77,0.497004,0.650000,0.375000,1.000000,0.461538,0.587535,0.500000,0.666667,0.482143,1.000000,0,40,16,1,26,0,6,3,56,2,11181.465808,13709.000000,8652.000000,10009.000000,11832.384615,11680.469576,9368.666667,10773.000000,10731.125000,9592.000000,2.360589,2.180000,1.720000,13.00,2.936923,2.362406,3.178333,1.890000,2.022321,1.170000,143.656231,174.800000,53.300000,27.000000,137.484615,175.621208,31.000000,112.000000,170.705357,15.000000,0.502224,0.520850,0.487365,0.574638,0.530157,0.581295,0.549502,0.534400,0.422799,0.534542,r
4,75,25,62,19,202,86,119,18,43,11,0.554367,0.333333,0.420000,0.600000,0.500000,0.000000,0.000000,0.650000,0.000000,0.545451,0,9,0,5,38,1,1,20,1,0,11401.275223,10597.444444,9072.000000,13082.000000,10994.000000,12009.000000,9005.000000,13383.000000,11146.000000,11850.221581,2.536424,2.653333,2.290000,1.63,2.790000,1.880000,2.570000,2.050000,2.270000,2.427233,156.077872,66.666667,105.200000,52.800000,141.800000,162.000000,76.000000,172.600000,23.000000,125.787012,0.553053,0.549502,0.420000,0.531027,0.534400,0.533874,0.520850,0.528621,0.519789,0.545546,b


First, we'll split the datasets into features and class labels.

data[name]_X will contain the features, and data[name]_Y will contain the class labels.

We will have to convert the class labels to binary for scikit-learn to be able to work with them. We will arbitrarily say 0 means the blue team wins, and 1 means the red team wins.

In [9]:
# data prep for data
data_Y = data['winner']
data_Y = data_Y.values.ravel()
data_X = data.drop(['winner'],axis=1)
print(data_X.head)
print()
print("data_Y pre-binarization: ", data_Y)

# binarize data_Y
binarizer = preprocessing.LabelBinarizer()
data_Y = binarizer.fit_transform(data_Y)
data_Y = np.rot90(data_Y)
data_Y = data_Y[0]
print("data_Y post-binarization: ", data_Y)

<bound method NDFrame.head of       b1_champ_id  ...  r_champ_player_win_diff
0              34  ...                 0.270394
1              23  ...                 0.440548
2              99  ...                -0.064419
3             122  ...                 0.613805
4              75  ...                -1.453228
...           ...  ...                      ...
2925          887  ...                 0.037825
2926          222  ...                 0.004268
2927          122  ...                 0.706182
2928           12  ...                 0.127274
2929           27  ...                 0.540623

[2930 rows x 84 columns]>

data_Y pre-binarization:  ['r' 'b' 'r' ... 'r' 'r' 'b']
data_Y post-binarization:  [1 0 1 ... 1 1 0]


In [13]:
# data prep for data_teams
data_teams_Y = data_teams['winner']
data_teams_Y = data_teams_Y.values.ravel()
data_teams_X = data_teams.drop(['winner'],axis=1)
print(data_teams_X.head)
print()
print("data_teams_Y pre-binarization: ", data_teams_Y)

# binarize data_teams_Y
binarizer = preprocessing.LabelBinarizer()
data_teams_Y = binarizer.fit_transform(data_teams_Y)
data_teams_Y = np.rot90(data_teams_Y)
data_teams_Y = data_teams_Y[0]
print("data_teams_Y post-binarization: ", data_teams_Y)

<bound method NDFrame.head of       b_win_sum  r_win_sum  ...  b_champ_player_win_diff  r_champ_player_win_diff
0      2.914206   2.931831  ...                 0.389538                 0.270394
1      2.759413   2.974894  ...                 0.138329                 0.440548
2      2.705163   2.491890  ...                 0.064698                -0.064419
3      2.983542   3.236344  ...                 0.368309                 0.613805
4      2.407700   1.195451  ...                -0.180283                -1.453228
...         ...        ...  ...                      ...                      ...
2925   2.509286   2.684566  ...                -0.038688                 0.037825
2926   2.792108   2.498352  ...                 0.222891                 0.004268
2927   2.557505   3.361648  ...                -0.087729                 0.706182
2928   1.594672   2.754835  ...                -0.920140                 0.127274
2929   2.993461   3.086309  ...                 0.434805            

In [12]:
# data prep for data_original
data_original_Y = data_original['winner']
data_original_Y = data_original_Y.values.ravel()
data_original_X = data_original.drop(['winner'],axis=1)
print(data_original_X.head)
print()
print("data_original_Y pre-binarization: ", data_original_Y)

# binarize data_original_Y
binarizer = preprocessing.LabelBinarizer()
data_original_Y = binarizer.fit_transform(data_original_Y)
data_original_Y = np.rot90(data_original_Y)
data_original_Y = data_original_Y[0]
print("data_original_Y post-binarization: ", data_original_Y)

<bound method NDFrame.head of       b1_champ_id  b2_champ_id  ...  r9_champ_win_rate  r10_champ_win_rate
0              34          203  ...           0.557449            0.518675
1              23           67  ...           0.434316            0.554777
2              99          222  ...           0.514967            0.420000
3             122          119  ...           0.422799            0.534542
4              75           25  ...           0.519789            0.545546
...           ...          ...  ...                ...                 ...
2925          887           63  ...           0.513538            0.547719
2926          222          875  ...           0.514967            0.420000
2927          122          105  ...           0.533896            0.554777
2928           12           39  ...           0.506626            0.529065
2929           27          234  ...           0.457430            0.496690

[2930 rows x 70 columns]>

data_original_Y pre-binarization:  ['r' 'b

Finally, we'll look at how a naive bayes classifier handles each dataset and compare results. We'll use 10-fold cross validation.

In [24]:
# combined dataset
clf = GaussianNB()
nested_score = cross_val_score(clf, data_X, data_Y, cv=10)
print("Accuracy:", nested_score.mean()*100)
print()

pred = cross_val_predict(clf, data_X, data_Y, cv=10)
cm = confusion_matrix(data_Y, pred)
print(cm)
print(classification_report(data_Y, pred))

Accuracy: 55.52901023890785

[[631 838]
 [465 996]]
              precision    recall  f1-score   support

           0       0.58      0.43      0.49      1469
           1       0.54      0.68      0.60      1461

    accuracy                           0.56      2930
   macro avg       0.56      0.56      0.55      2930
weighted avg       0.56      0.56      0.55      2930



In [25]:
# feature engineered dataset
clf = GaussianNB()
nested_score = cross_val_score(clf, data_teams_X, data_teams_Y, cv=10)
print("Accuracy:", nested_score.mean()*100)
print()

pred = cross_val_predict(clf, data_teams_X, data_teams_Y, cv=10)
cm = confusion_matrix(data_teams_Y, pred)
print(cm)
print(classification_report(data_teams_Y, pred))

Accuracy: 58.97610921501707

[[751 718]
 [484 977]]
              precision    recall  f1-score   support

           0       0.61      0.51      0.56      1469
           1       0.58      0.67      0.62      1461

    accuracy                           0.59      2930
   macro avg       0.59      0.59      0.59      2930
weighted avg       0.59      0.59      0.59      2930



In [26]:
# original features dataset
clf = GaussianNB()
nested_score = cross_val_score(clf, data_original_X, data_original_Y, cv=10)
print("Accuracy:", nested_score.mean()*100)
print()

pred = cross_val_predict(clf, data_original_X, data_original_Y, cv=10)
cm = confusion_matrix(data_original_Y, pred)
print(cm)
print(classification_report(data_original_Y, pred))

Accuracy: 53.78839590443685

[[593 876]
 [478 983]]
              precision    recall  f1-score   support

           0       0.55      0.40      0.47      1469
           1       0.53      0.67      0.59      1461

    accuracy                           0.54      2930
   macro avg       0.54      0.54      0.53      2930
weighted avg       0.54      0.54      0.53      2930



Based on these results, we see that the original features dataset ranks lowest in accuracy, and that the dataset with only the engineered features ranked highest in accuracy. This was expected because, as previously discussed, each team member listed is in an arbitrary position in the dataset (i.e. there is no meaningful systematic difference between player r4 and player r5; they could be switched without a problem). Therefore, combining these arbitrary features into either an average or sum for each team improved accuracy for the classifier.